In [1]:
#Dependencies
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import matplotlib.pyplot as plt
import pymysql
pymysql.install_as_MySQLdb()

In [2]:
#Import Datasets
#Player Data
player_file = "./Data/Players.csv"
player_df = pd.read_csv(player_file)
#College State Data
college_file = "./Data/hd2017.csv"
college_df = pd.read_csv(college_file, encoding='latin-1')
#Dataset with state names and their abreviations
state_file = "./Data/state_abv.csv"
state_df = pd.read_csv(state_file)

In [3]:
#View Player Dataframe
player_df.head()

,Unnamed: 0,Player,height,weight,collage,born,birth_city,birth_state
0,0,Curly Armstrong,180.0,77.0,Indiana University,1918.0,NaN,NaN
1,1,Cliff Barker,188.0,83.0,University of Kentucky,1921.0,Yorktown,Indiana
2,2,Leo Barnhorst,193.0,86.0,University of Notre Dame,1924.0,NaN,NaN
3,3,Ed Bartels,196.0,88.0,North Carolina State University,1925.0,NaN,NaN
4,4,Ralph Beard,178.0,79.0,University of Kentucky,1927.0,Hardinsburg,Kentucky


In [4]:
#View State Dataframe
college_df.head()

,UNITID,INSTNM,IALIAS,ADDR,CITY,STABBR,ZIP,FIPS,OBEREG,CHFNM,...,CBSATYPE,CSA,NECTA,COUNTYCD,COUNTYNM,CNGDSTCD,LONGITUD,LATITUDE,DFRCGID,DFRCUSCG
0,100654,Alabama A & M University,AAMU,4900 Meridian Street,Normal,AL,35762,1,5,"Dr. Andrew Hugine, Jr.",...,1,290,-2,1089,Madison County,105,-86.568502,34.783368,122,1
1,100663,University of Alabama at Birmingham,,Administration Bldg Suite 1070,Birmingham,AL,35294-0110,1,5,Ray L. Watts,...,1,142,-2,1073,Jefferson County,107,-86.799345,33.505697,109,1
2,100690,Amridge University,Southern Christian University |Regions University,1200 Taylor Rd,Montgomery,AL,36117-3553,1,5,Michael C.Turner,...,1,-2,-2,1101,Montgomery County,102,-86.174010,32.362609,141,2
3,100706,University of Alabama in Huntsville,UAH |University of Alabama Huntsville,301 Sparkman Dr,Huntsville,AL,35899,1,5,Robert A. Altenkirch,...,1,290,-2,1089,Madison County,105,-86.640449,34.724557,112,2
4,100724,Alabama State University,,915 S Jackson Street,Montgomery,AL,36104-0271,1,5,Quinton T. Ross,...,1,-2,-2,1101,Montgomery County,107,-86.295677,32.364317,131,1


In [5]:
#View State Dataframe
state_df.head()

,State,Abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [6]:
#Dropping unnecessary columns and renaming mispelled columns from player_df
clean_player_df = player_df[['Player', 'collage', 'birth_state']].copy()
clean_player_df = clean_player_df.rename(columns={'Player':'player','collage':'college'})
clean_player_df.head()

,player,college,birth_state
0,Curly Armstrong,Indiana University,NaN
1,Cliff Barker,University of Kentucky,Indiana
2,Leo Barnhorst,University of Notre Dame,NaN
3,Ed Bartels,North Carolina State University,NaN
4,Ralph Beard,University of Kentucky,Kentucky


In [7]:
#Dropping unnecessary columns and renaming  columns from college_df
clean_college_df = college_df[['INSTNM', 'STABBR']].copy()
clean_college_df = clean_college_df.rename(columns={'INSTNM': 'college', 'STABBR': 'Abbreviation'})
clean_college_df.head()

,college,Abbreviation
0,Alabama A & M University,AL
1,University of Alabama at Birmingham,AL
2,Amridge University,AL
3,University of Alabama in Huntsville,AL
4,Alabama State University,AL


In [8]:
#Merging state and college dataframes and dropping abbreviation column
clean_college_df = clean_college_df.merge(state_df, on='Abbreviation')
clean_college_df = clean_college_df.drop(columns='Abbreviation')
clean_college_df.head()

,college,State
0,Alabama A & M University,Alabama
1,University of Alabama at Birmingham,Alabama
2,Amridge University,Alabama
3,University of Alabama in Huntsville,Alabama
4,Alabama State University,Alabama


In [9]:
#Merging college and player dataframes
clean_player_df = clean_player_df.merge(clean_college_df, on="college")
clean_player_df = clean_player_df.rename(columns={"State":"college_state"})
clean_player_df = clean_player_df.merge(state_df, how="left", left_on="birth_state", right_on="State")
clean_player_df.head()

,player,college,birth_state,college_state,State,Abbreviation
0,Cliff Barker,University of Kentucky,Indiana,Kentucky,Indiana,IN
1,Ralph Beard,University of Kentucky,Kentucky,Kentucky,Kentucky,KY
2,George Feigenbaum,University of Kentucky,NaN,Kentucky,NaN,NaN
3,Alex Groza,University of Kentucky,Ohio,Kentucky,Ohio,OH
4,Joe Holland,University of Kentucky,NaN,Kentucky,NaN,NaN


In [10]:
#Dropping NaNs and and State and Abbreviation columns
clean_player_df = clean_player_df.dropna()
clean_player_df = clean_player_df.drop(columns)
clean_player_df.head()

,player,college,birth_state,college_state,State,Abbreviation
0,Cliff Barker,University of Kentucky,Indiana,Kentucky,Indiana,IN
1,Ralph Beard,University of Kentucky,Kentucky,Kentucky,Kentucky,KY
3,Alex Groza,University of Kentucky,Ohio,Kentucky,Ohio,OH
5,Paul Noel,University of Kentucky,Kentucky,Kentucky,Kentucky,KY
6,Jack Parkinson,University of Kentucky,Indiana,Kentucky,Indiana,IN


In [13]:
clean_player_df = clean_player_df.drop(columns=['State', 'Abbreviation'])
clean_player_df.head()

,player,college,birth_state,college_state
0,Cliff Barker,University of Kentucky,Indiana,Kentucky
1,Ralph Beard,University of Kentucky,Kentucky,Kentucky
3,Alex Groza,University of Kentucky,Ohio,Kentucky
5,Paul Noel,University of Kentucky,Kentucky,Kentucky
6,Jack Parkinson,University of Kentucky,Indiana,Kentucky


In [14]:
#Saving to csv
clean_player_df.to_csv('nba_complete.csv')

In [15]:
#Creating Database Connection
connection_string = "root:S2hockey@localhost/ncaa_db"
engine = create_engine(f'mysql://{connection_string}')

In [16]:
#Confirming table name
engine.table_names()

['geo_states', 'ncaa_info']

In [17]:
#Load data into SQL database
clean_player_df.to_sql(name='ncaa_info', con=engine, if_exists='append', index=False)